## Author Profiling in Twitter. Gender Prediction.

### The code below has been used to classify tweets written in Spanish language. Statistical (such as Tf-Idf) and linguistic (such as POS Tagging) features will be used to increase the provided Baseline accuracy of 0.6643. The following code builds the best of the classifiers that has been found. Ths code does not reflect all testing that has been done under the scope of the task.

### Loading libraries that will be used in this notebook.

In [2]:
import nltk
import sklearn
import pandas
print("NLTK version: ", nltk.__version__)
print("Scikit-learn version: ", sklearn.__version__)
print("Pandas version: ", pandas.__version__)
import csv
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model
from scipy.sparse import hstack
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import re

NLTK version:  3.2.3
Scikit-learn version:  0.18.1
Pandas version:  0.20.1


### Tweets for train and test will be loaded form a text files. Accents have been kept for Gender prediction since testing has shown the prediction is better when accents are not removed.

### Loading Train and Test Tweets to panda's dataframes.

In [3]:
file_train = "train_tweets.csv"
names = ['ID','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']
df_train = pandas.read_csv(file_train,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [3]:
file_test = "test_tweets.csv"
names = ['ID','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']
df_test = pandas.read_csv(file_test,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

### Showing how the train and test dataframes look like.

In [4]:
df_train.head(1)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T91,T92,T93,T94,T95,T96,T97,T98,T99,T100
ID,,,,,,,,,,,,,,,,,,,,,
fff46823954870db83b3e6c74a60412c,Una practica en el Arbitraje CIADI es que si h...,@tromepe Esto me parece una falta de respeto p...,Muy buena carlincatura. https://t.co/jmYFN4rZe1,"En estos tiempos de calor infernal, ser bomber...",Estoy cansado de leer que congreso solicita co...,@Politica_ECpe @elcomercio Sres. Tengan en cla...,@DiarioAltavoz Una historia con un mismo villa...,"@DiarioAltavoz Fiscalía vs. Congreso, es una l...",@noticiAmerica Y de que manera ayudaremos si p...,@ammy_rocks24 te hablan creo Dolly. En coquete...,...,"Dicen que todos los cambios son buenos, a empe...",@noticiAmerica Hubo show para toda clase de pu...,Para algunos este discurso fue mas de lo mismo...,Excelente participación la del congresista Fra...,"Dios te da tres respuestas: Si, no o espera. B...",Como sea queria casarse. https://t.co/qxQxs6Jlro,El mundo ya se perdio. https://t.co/04G71M0fj5,Le quedo grande el papel de Guasón a Jared Let...,@Luisdiazc21 una caracteristica tuya mi buen a...,"Macho peruano que se respeta, toma chela helad..."


In [5]:
df_test.head(1)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T91,T92,T93,T94,T95,T96,T97,T98,T99,T100
ID,,,,,,,,,,,,,,,,,,,,,
ffb5c29c835e3cecbdaa97bfea5bbe3b,Que se haga tu voluntad y no la mía Señor... S...,A congregarse!!!LEVANTATE... DEJA LAS EXCUSAS ...,EVANGELISTA JUAN CARLOS ACEVEDO PFEIFFER https...,"... La lengua, miembro pequeño del cuerpo pero...",13. Por eso en circunstancias como éstas guard...,"ANTE LA IGNORANCIA, LA ESTUPIDEZ Y LA INSOLENC...",Si hay algo que nunca... jamás... never... no ...,No preguntes porqué?... Deja simplemente que e...,Mi deber está en permitir que se haga su volun...,EVANGELISTA JUAN CARLOS ACEVEDO PFEIFFER https...,...,'Can= Puedo VS. Can''t= No puedoJESUCRISTO SI ...,... En ti tengo paz!!!EVANGELISTA JUAN CARLOS ...,SOY UN GUERRERO DEL DIOS VIVIENTE Y JEHOVÁ DE ...,1. En ese momento los discípulos se acercaron ...,Cristo resucitó... Los demás \no\!EN QUIEN CRE...,... No cometas ese error... Te arrepentirás!!!...,"13. »Por tanto —agregó Jesús—, manténganse des...","38. Den, y se les dará: se les echará en el re...",... Dar gracias al Señor en todo tiempo!!!AMÉN...,"1. Señor , oye mi justo ruego; escucha mi clam..."


### Loading Train and Test labels.

In [6]:
file_labels_train = "train_classes.csv"
names = ['ID','Variety','Gender']
df_train_labels = pandas.read_csv(file_labels_train,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [7]:
file_labels_test = "test_classes.csv"
names = ['ID','Variety','Gender']
df_test_labels = pandas.read_csv(file_labels_test,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

### Showing how the label dataframes look like. Two different labels can be found for each tweet's author, which classify it in Variety (country the author comes from) and Gender (either the author is male or female).

In [8]:
df_train_labels.head(1)

,Variety,Gender
ID,,
fff46823954870db83b3e6c74a60412c,peru,male


In [9]:
df_test_labels.head(1)

,Variety,Gender
ID,,
ffb5c29c835e3cecbdaa97bfea5bbe3b,colombia,male


### Tweets' text pre-processing. Text in tweets will be converted to lowercase, tokenized using Twitter tokenizer and spanish stop-words will be removed from the text.

In [10]:
tknzr = TweetTokenizer()

#### Text to lowercase.

In [11]:
df_train = df_train.applymap(lambda x: x.lower())

In [12]:
df_test = df_test.applymap(lambda x: x.lower())

#### A list is built in order to being able to access tweet per user.

In [13]:
lista_tweets = ['T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']

#### Twitter Tokenization.

In [14]:
for item in lista_tweets:
    df_train[item] = df_train.apply(lambda row: tknzr.tokenize(row[item]),axis=1)

In [15]:
for item in lista_tweets:
    df_test[item] = df_test.apply(lambda row: tknzr.tokenize(row[item]),axis=1)

#### Spanish stop-words removal.

In [16]:
stop = stopwords.words('spanish')

In [17]:
for item in lista_tweets:
    df_train[item] = df_train[item].apply(lambda tweet: [word for word in tweet if word not in stop])

In [18]:
for item in lista_tweets:
    df_test[item] = df_test[item].apply(lambda tweet: [word for word in tweet if word not in stop])

#### Packing tweets together. Tweets have to be de-tokenized before being submitted to following processing steps.

In [19]:
for item in lista_tweets:
    df_train[item] = df_train[item].apply(lambda x: " ".join(str(z) for z in x))

In [20]:
for item in lista_tweets:
    df_test[item] = df_test[item].apply(lambda x: " ".join(str(z) for z in x))

### Additional features creation.

### Some additonal features will be created from the tweet's text. Linguistic features like Part Of Speech tags, and several statistical features such us word count, and Twitter specific items counting (hashtags, and more)

### POS-Tagger. Stanford POS Tagger has been run for both the Train and Test datasets. Resuts have been written to csv files, which will be loaded here.

In [21]:
file_train_tag = "tagged_train_filtered.csv"
names = ['ID','Adj','Conj','Det','Punc','Int','Noun','Pron','Adv','Prep','Verb','Date','Num']
tag_df_train = pandas.read_csv(file_train_tag,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [22]:
file_test_tag = "tagged_test_filtered.csv"
names = ['ID','Adj','Conj','Det','Punc','Int','Noun','Pron','Adv','Prep','Verb','Date','Num']
tag_df_test = pandas.read_csv(file_test_tag,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

### POS tag's values will be standarized in order to get values in range [0..1], which are optimal for machine learning methodologies.

In [23]:
tag_df_train["Adj"]=tag_df_train["Adj"]/max(tag_df_train["Adj"])
tag_df_train["Conj"]=tag_df_train["Conj"]/max(tag_df_train["Conj"])
tag_df_train["Det"]=tag_df_train["Det"]/max(tag_df_train["Det"])
tag_df_train["Punc"]=tag_df_train["Punc"]/max(tag_df_train["Punc"])
tag_df_train["Int"]=tag_df_train["Int"]/max(tag_df_train["Int"])
tag_df_train["Noun"]=tag_df_train["Noun"]/max(tag_df_train["Noun"])
tag_df_train["Pron"]=tag_df_train["Pron"]/max(tag_df_train["Pron"])
tag_df_train["Adv"]=tag_df_train["Adv"]/max(tag_df_train["Adv"])
tag_df_train["Prep"]=tag_df_train["Prep"]/max(tag_df_train["Prep"])
tag_df_train["Verb"]=tag_df_train["Verb"]/max(tag_df_train["Verb"])
tag_df_train["Date"]=tag_df_train["Date"]/max(tag_df_train["Date"])
tag_df_train["Num"]=tag_df_train["Num"]/max(tag_df_train["Num"])

In [24]:
tag_df_test["Adj"]=tag_df_test["Adj"]/max(tag_df_test["Adj"])
tag_df_test["Conj"]=tag_df_test["Conj"]/max(tag_df_test["Conj"])
tag_df_test["Det"]=tag_df_test["Det"]/max(tag_df_test["Det"])
tag_df_test["Punc"]=tag_df_test["Punc"]/max(tag_df_test["Punc"])
tag_df_test["Int"]=tag_df_test["Int"]/max(tag_df_test["Int"])
tag_df_test["Noun"]=tag_df_test["Noun"]/max(tag_df_test["Noun"])
tag_df_test["Pron"]=tag_df_test["Pron"]/max(tag_df_test["Pron"])
tag_df_test["Adv"]=tag_df_test["Adv"]/max(tag_df_test["Adv"])
tag_df_test["Prep"]=tag_df_test["Prep"]/max(tag_df_test["Prep"])
tag_df_test["Verb"]=tag_df_test["Verb"]/max(tag_df_test["Verb"])
tag_df_test["Date"]=tag_df_test["Date"]/max(tag_df_test["Date"])
tag_df_test["Num"]=tag_df_test["Num"]/max(tag_df_test["Num"])

### In the boxes below several tags can be selected to be removed form the POS tags feature matrix, so they will not be taken into account when building the classifier. During the testing process this has been useful to find the best classifier.

In [ ]:
#del(tag_df_train["Conj"])
#del(tag_df_train["Det"])
#del(tag_df_train["Punc"])
#del(tag_df_train["Int"])
#del(tag_df_train["Noun"])
#del(tag_df_train["Pron"])
#del(tag_df_train["Adv"])
#del(tag_df_train["Prep"])
#del(tag_df_train["Verb"])
#del(tag_df_train["Date"])
#del(tag_df_train["Num"])

In [20]:
#del(tag_df_test["Conj"])
#del(tag_df_test["Det"])
#del(tag_df_test["Punc"])
#del(tag_df_test["Int"])
#del(tag_df_test["Noun"])
#del(tag_df_test["Pron"])
#del(tag_df_test["Adv"])
#del(tag_df_test["Prep"])
#del(tag_df_test["Verb"])
#del(tag_df_test["Date"])
#del(tag_df_test["Num"])

### Twitter specific features.

#### Counting emojis per author. The Tweet's text is gone over to count how many emojis every author uses.

In [19]:
emojis_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'[\U0001f600-\U0001f650]', aux2)))
    emojis_tweets_train.append(cuenta)

In [20]:
emojis_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'[\U0001f600-\U0001f650]', aux2)))
    emojis_tweets_test.append(cuenta)

In [21]:
tag_df_train["Emojis"]=emojis_tweets_train
tag_df_test["Emojis"]=emojis_tweets_test

#### Again, the counts are standarized [0..1] by dividing by the max value.

In [22]:
tag_df_train["Emojis"] = tag_df_train["Emojis"]/max(tag_df_train["Emojis"])
tag_df_test["Emojis"] = tag_df_test["Emojis"]/max(tag_df_test["Emojis"])

#### Hashtags counter

In [23]:
hashtags_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", aux2)))
    hashtags_tweets_train.append(cuenta)

In [24]:
hashtags_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", aux2)))
    hashtags_tweets_test.append(cuenta)

In [25]:
tag_df_train["Hashtags"]=hashtags_tweets_train
tag_df_test["Hashtags"]=hashtags_tweets_test

In [26]:
tag_df_train["Hashtags"] = tag_df_train["Hashtags"]/max(tag_df_train["Hashtags"])
tag_df_test["Hashtags"] = tag_df_test["Hashtags"]/max(tag_df_test["Hashtags"])

#### Mentions counter.

In [27]:
mentions_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'(?:@[\w_]+)', aux2)))
    mentions_tweets_train.append(cuenta)

In [28]:
mentions_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'(?:@[\w_]+)', aux2)))
    mentions_tweets_test.append(cuenta)

In [29]:
tag_df_train["Mentions"]=mentions_tweets_train
tag_df_test["Mentions"]=mentions_tweets_test

In [30]:
tag_df_train["Mentions"] = tag_df_train["Mentions"]/max(tag_df_train["Mentions"])
tag_df_test["Mentions"] = tag_df_test["Mentions"]/max(tag_df_test["Mentions"])

#### URLs counter.

In [27]:
urls_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', aux2)))
    urls_tweets_train.append(cuenta)

In [28]:
urls_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', aux2)))
    urls_tweets_test.append(cuenta)

In [29]:
tag_df_train["URLs"]=urls_tweets_train
tag_df_test["URLs"]=urls_tweets_test

In [30]:
tag_df_train["URLs"] = tag_df_train["URLs"]/max(tag_df_train["URLs"])
tag_df_test["URLs"] = tag_df_test["URLs"]/max(tag_df_test["URLs"])

#### Counting retwitting.

In [27]:
rts_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r"(RT|via)((?:\b\W*@\w+)+)", aux2)))
    rts_tweets_train.append(cuenta)

In [28]:
rts_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        cuenta = cuenta + (len(re.findall(r"(RT|via)((?:\b\W*@\w+)+)", aux2)))
    rts_tweets_test.append(cuenta)

In [29]:
tag_df_train["RTs"]=rts_tweets_train
tag_df_test["RTs"]=rts_tweets_test

In [30]:
tag_df_train["RTs"] = tag_df_train["RTs"]/max(tag_df_train["RTs"])
tag_df_test["RTs"] = tag_df_test["RTs"]/max(tag_df_test["RTs"])

In [87]:
tag_df_train.head(5)

,Adj,Conj,Det,Punc,Int,Noun,Pron,Adv,Prep,Verb,Date,Num,Emojis,Hashtags,Mentions,URLs,RTs
ID,,,,,,,,,,,,,,,,,
fff46823954870db83b3e6c74a60412c,0.338583,0.446512,0.537572,0.273247,0.265306,0.345794,0.356643,0.295455,0.420925,0.499058,0.072917,0.018373,0.000000,0.021645,0.164927,0.161616,0.0
ffebe1735cd1f0e69d8210376a9dc377,0.322835,0.204651,0.416185,0.284692,0.102041,0.369584,0.188811,0.245455,0.559611,0.310734,0.114583,0.026247,0.000000,0.149351,0.033403,0.515152,0.0
ffe2faddf807b45ade8d8032ab7487d4,0.326772,0.120930,0.320809,0.138770,0.010204,0.369584,0.108392,0.095455,0.416058,0.259887,0.052083,0.023622,0.000000,0.058442,0.002088,0.898990,0.0
ffc9c0b137c5bbb3f9173e7af991e122,0.248031,0.186047,0.280347,0.211731,0.102041,0.361088,0.300699,0.381818,0.218978,0.367232,0.239583,0.041995,0.050505,0.025974,0.728601,0.348485,0.0
ff91e621bd80e9c980a6e7f8550a1d80,0.464567,0.534884,0.595376,0.472103,0.173469,0.356839,0.524476,0.718182,0.547445,0.580038,0.072917,0.049869,0.000000,0.019481,0.108559,0.060606,0.0


#### Word count per user. This is a linguistic feature that will be added to the feature's dataframe. 

In [25]:
words_tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        pals = tknzr.tokenize(aux2)
        cuenta = cuenta + len(pals)
    words_tweets_train.append(cuenta)

In [26]:
words_tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i]
    cuenta = 0
    for item in lista_tweets:
        aux2= aux[item]
        pals = tknzr.tokenize(aux2)
        cuenta = cuenta + len(pals)
    words_tweets_test.append(cuenta)

In [27]:
tag_df_train["Words"]=words_tweets_train
tag_df_test["Words"]=words_tweets_test

In [28]:
tag_df_train["Words"] = tag_df_train["Words"]/max(tag_df_train["Words"])
tag_df_test["Words"] = tag_df_test["Words"]/max(tag_df_test["Words"])

## Modeling and evaluating.

### Loading labels for Train and Test.

In [29]:
label_gender_train = df_train_labels["Gender"].values.tolist()

In [30]:
label_gender_test = df_test_labels["Gender"].values.tolist()

### All 100 Tweets per author have to be joined in a single string in order to use Tf-Idf Vectorizer.

### Grouping all tweets per author (train and test)

In [4]:
tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i].values.tolist()
    tweets_train.append(' '.join(aux))

In [32]:
tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i].values.tolist()
    tweets_test.append(' '.join(aux))

#### During the testing it has been found that setting max_df parametre to 0.90 provides the best accuracy. max_df sets the maximum word fequency to be kept. That means that words with a frequency higher than 0.90 will be considered as being corpus specific stop-words and therefore will be removed since they do not contribute positively to the model. 

### Tf-Idf Unigrams + corpus stop-words removal (max_df=0.90)

In [47]:
TfIdf_uni_df=TfidfVectorizer(analyzer="word",lowercase=False, ngram_range=(1,1), max_df=0.90, min_df=0)

In [48]:
train_tfidf_uni_df = TfIdf_uni_df.fit_transform(tweets_train)

In [49]:
test_tfidf_uni_df = TfIdf_uni_df.transform(tweets_test)

## Modeling Tf-Idf + features dataframe

#### Features dataframe is loaded as a matrix, both for train and test.

In [36]:
df_train_feat_to_matrix = tag_df_train.as_matrix()

In [37]:
df_test_feat_to_matrix = tag_df_test.as_matrix()

#### Sparse matrices obtained as a result of Tf-Idf are joint with features matrices.

In [38]:
dades_train_tf =hstack([train_tfidf_uni_df, df_train_feat_to_matrix]).toarray()

In [39]:
dades_test_tf =hstack([test_tfidf_uni_df, df_test_feat_to_matrix]).toarray()

#### Multi-Layer Perceptron model creation.

In [40]:
ann_gen = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,), random_state=1)

#### Fitting the train dataset, including Tf-Idf and created features, to the train labels.

In [41]:
ann_gen.fit(dades_train_tf, label_gender_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

#### Test dataset's labels prediction.

In [43]:
pred_ann_gen = ann_gen.predict(dades_test_tf)

#### Calculating accuracy and f1-score measures for the model using scikitlearn's metrics package.

In [44]:
acc_tfidf_gen_feat_ann = accuracy_score(label_gender_test, pred_ann_gen)
f1_tfidf_gen_feat_ann = metrics.f1_score(label_gender_test, pred_ann_gen, average='weighted')

In [45]:
print ("ANN Gender Accuracy amb features + Tf-Idf= ",str(acc_tfidf_gen_feat_ann))
print ("ANN Gender f1-score amb features + Tf-Idf= ",str(f1_tfidf_gen_feat_ann))

ANN Gender Accuracy amb features + Tf-Idf=  0.774285714286
ANN Gender f1-score amb features + Tf-Idf=  0.772670748727
